In [137]:
import collections

In [77]:
import sys
import numpy as np

from astropy import units as u
from astropy import table, modeling

from scipy import stats

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import __version__ as mpl_vers
mpl_vers

'3.0.0'

In [3]:
pathtoinsert = '..'  # should have emceemr
if pathtoinsert not in sys.path:
    sys.path.insert(1, pathtoinsert)

import emceemr

In [4]:
import inspect

In [5]:
inspect.getmro(modeling.models.Gaussian1D)

(<class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev'),
 <class 'astropy.modeling.core.Fittable1DModel'>,
 <class 'astropy.modeling.core.FittableModel'>,
 <class 'astropy.modeling.core.Model'>,
 object)

In [61]:
class ProbabilisticModel(modeling.Model):
    pass

class Test1(ProbabilisticModel, modeling.models.Gaussian1D):
    _inherit_descriptors_ = (modeling.Parameter,)
    proby = modeling.Parameter(default=0.5)
    
class Test2(modeling.models.Gaussian1D, ProbabilisticModel):
    _inherit_descriptors_ = (modeling.Parameter,)
    proby = modeling.Parameter(default=0.5)

Test1._parameters_, Test2._parameters_

(OrderedDict([('proby', Parameter('proby', default=0.5))]),
 OrderedDict([('amplitude', Parameter('amplitude', default=1)),
              ('mean', Parameter('mean', default=0)),
              ('stddev',
               Parameter('stddev', default=1, bounds=(1.1754943508222875e-38, None))),
              ('proby', Parameter('proby', default=0.5))]))

In [62]:
class ProbabilisticModel(modeling.Model):
    _inherit_descriptors_ = (modeling.Parameter,)

class Test1(ProbabilisticModel, modeling.models.Gaussian1D):
    proby = modeling.Parameter(default=0.5)
    
class Test2(modeling.models.Gaussian1D, ProbabilisticModel):
    proby = modeling.Parameter(default=0.5)

Test1._parameters_, Test2._parameters_

(OrderedDict([('amplitude', Parameter('amplitude', default=1)),
              ('mean', Parameter('mean', default=0)),
              ('stddev',
               Parameter('stddev', default=1, bounds=(1.1754943508222875e-38, None))),
              ('proby', Parameter('proby', default=0.5))]),
 OrderedDict([('amplitude', Parameter('amplitude', default=1)),
              ('mean', Parameter('mean', default=0)),
              ('stddev',
               Parameter('stddev', default=1, bounds=(1.1754943508222875e-38, None))),
              ('proby', Parameter('proby', default=0.5))]))

In [63]:
class ProbabilisticModel(modeling.Model):
    _inherit_descriptors_ = (modeling.Parameter,)
    
    
    

Base API: ProbabilisticModel should have a mean model and a noise model.

Noise Model should take the mean model + data and output a log-prob of the same shape as the mean model output.  Form can be one of
 * modeling model that returns log-prob given whole mean-model (one of the parameters is the mean-model part)?
 * modeling model that returns prob given whole mean-model (one of the parameters is the mean-model part)?
 * scipy.stats probability distribution. need to figure out how to get parameters...
 * callable that takes in log-prob
    
For experimentation, use a linear mean model and gaussian noise model

In [76]:
ndata = 100

mean_model = modeling.models.Linear1D(slope=2, intercept=1)
noise_model = modeling.models.Gaussian1D(mean = np.random.randn(ndata))

In [103]:
stats.beta.shapes

'a, b'

In [132]:
f = stats.beta(1, 2, scale=3)
f.dist._parse_args(*f.args, **f.kwds)

((1, 2), 0, 3)

In [134]:
f = stats.alpha(2, loc=3)
f.dist._parse_args(*f.args, **f.kwds)

((2,), 3, 1)

In [262]:
stats.norm.logpdf(0, loc=1)

-1.4189385332046727

In [260]:
Mod.name


'BetaLogPDFModel'

In [237]:
def generate_logpdfmodel_class(frozen_distribution, scale_param_name='prob_scale', location_param_name='prob_loc', data_param_name='prob_loc'):
    dist = frozen_distribution.dist
    other_params, loc, scale = dist._parse_args(*frozen_distribution.args, **frozen_distribution.kwds)
    other_param_names = [] if dist.shapes is None else dist.shapes.split(', ')

    assert len(other_params) == len(other_param_names), 'This stats object seems messed up - the names of the parameters do not match the number of parameters'

    # order matters here!  most go "other parameters in order, loc, scale"
    params = []
    for nm, p in zip(other_param_names, other_params):
        params.append(modeling.Parameter(name=nm, default=p))
    params.append(modeling.Parameter(name=location_param_name, default=loc, description='distribution location parameter'))
    params.append(modeling.Parameter(name=scale_param_name, default=scale, description='distribution scale parameter'))

    params = collections.OrderedDict(((p.name, p) for p in params))
    
    members = {}
    
    def tester(*args, **kwargs):
        print(args, kwargs)
        return dist.logpdf(*args, **kwargs)
        
    
    mod = find_current_module(1)
    modname = mod.__name__ if mod else '__main__'
    
    members = {
        '__module__': str(modname),
        'distribution': dist,
        'inputs': ('x',),
        'outputs': ('logpdf',),
        'evaluate': staticmethod(dist.logpdf),
    }
    members.update(params)
    
    return type(dist.name.title() + 'LogPDFModel', (modeling.FittableModel,), members)
    
    
Mod = generate_logpdfmodel_class(stats.beta(1,2, scale=3))
Mod

<class '__main__.BetaLogPDFModel'>
Name: BetaLogPDFModel
Inputs: ('x',)
Outputs: ('logpdf',)
Fittable parameters: ('a', 'b', 'prob_loc', 'prob_scale')

In [240]:
mod = Mod()
mod(1), stats.beta.logpdf(1, 1, 2, scale=3)

(-0.8109302162163288, -0.8109302162163288)

In [257]:
Mod2 = generate_logpdfmodel_class(stats.norm(scale=1), scale_param_name='sigma')
Mod2

<class '__main__.NormLogPDFModel'>
Name: NormLogPDFModel
Inputs: ('x',)
Outputs: ('logpdf',)
Fittable parameters: ('prob_loc', 'sigma')

In [258]:
mod = Mod2(prob_loc=[0, 1])
mod(0)

array([-0.91893853, -1.41893853])